In [16]:
import pandas as pd
import numpy as np

df = pd.read_excel("Adidas US Sales Datasets.xlsx", engine="openpyxl")

df.rename(columns=lambda c: c.strip(), inplace=True)

df['Invoice Date'] = pd.to_datetime(df['Invoice Date'], errors='coerce')
for col in ['Price per Unit','Units Sold','Total Sales','Operating Profit','Operating Margin']:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')

df['Product Category'] = df['Product'].str.split('-', n=1).str[0].str.strip()

df['Product Category'] = df['Product Category'].fillna(df['Product'])
df


,Retailer,Retailer ID,Invoice Date,Region,State,City,Product,Price per Unit,Units Sold,Total Sales,Operating Profit,Operating Margin,Sales Method,Product Category
0,Foot Locker,1185732,2020-01-01,Northeast,New York,New York,Men's Street Footwear,50.0,1200,600000.0,300000.00,0.50,In-store,Men's Street Footwear
1,Foot Locker,1185732,2020-01-02,Northeast,New York,New York,Men's Athletic Footwear,50.0,1000,500000.0,150000.00,0.30,In-store,Men's Athletic Footwear
2,Foot Locker,1185732,2020-01-03,Northeast,New York,New York,Women's Street Footwear,40.0,1000,400000.0,140000.00,0.35,In-store,Women's Street Footwear
3,Foot Locker,1185732,2020-01-04,Northeast,New York,New York,Women's Athletic Footwear,45.0,850,382500.0,133875.00,0.35,In-store,Women's Athletic Footwear
4,Foot Locker,1185732,2020-01-05,Northeast,New York,New York,Men's Apparel,60.0,900,540000.0,162000.00,0.30,In-store,Men's Apparel
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9643,Foot Locker,1185732,2021-01-24,Northeast,New Hampshire,Manchester,Men's Apparel,50.0,64,3200.0,896.00,0.28,Outlet,Men's Apparel
9644,Foot Locker,1185732,2021-01-24,Northeast,New Hampshire,Manchester,Women's Apparel,41.0,105,4305.0,1377.60,0.32,Outlet,Women's Apparel
9645,Foot Locker,1185732,2021-02-22,Northeast,New Hampshire,Manchester,Men's Street Footwear,41.0,184,7544.0,2791.28,0.37,Outlet,Men's Street Footwear
9646,Foot Locker,1185732,2021-02-22,Northeast,New Hampshire,Manchester,Men's Athletic Footwear,42.0,70,2940.0,1234.80,0.42,Outlet,Men's Athletic Footwear


**Q1 — Total revenue generated by each product category**

In [4]:
# Q1
res_q1 = df.groupby('Product Category', observed=False)['Total Sales'].sum().reset_index().sort_values('Total Sales', ascending=False)
res_q1


,Product Category,Total Sales
2,Men's Street Footwear,208826244.0
3,Women's Apparel,179038860.0
1,Men's Athletic Footwear,153673680.0
5,Women's Street Footwear,128002813.0
0,Men's Apparel,123728632.0
4,Women's Athletic Footwear,106631896.0


**Q2 — Sales method (In-store vs Outlet) that generates the highest revenue**

In [5]:
# Q2
res_q2 = df.groupby('Sales Method', observed=False)['Total Sales'].sum().reset_index().sort_values('Total Sales', ascending=False)
res_q2


,Sales Method,Total Sales
0,In-store,356643750.0
2,Outlet,295585493.0
1,Online,247672882.0


**Q3 — Average operating margin across all product categories**

In [6]:
# Q3
# Use Operating Margin column; drop NaNs to get meaningful averages
res_q3 = df.groupby('Product Category', observed=False)['Operating Margin'].mean().reset_index().sort_values('Operating Margin', ascending=False)
res_q3


,Product Category,Operating Margin
2,Men's Street Footwear,0.446130
3,Women's Apparel,0.441318
4,Women's Athletic Footwear,0.424359
0,Men's Apparel,0.413225
5,Women's Street Footwear,0.410199
1,Men's Athletic Footwear,0.402702


**Q4 — How total revenue changes across months (temporal trend)**

In [7]:
# Q4
df['YearMonth'] = df['Invoice Date'].dt.to_period('M')
res_q4 = df.groupby('YearMonth', observed=False)['Total Sales'].sum().reset_index().sort_values('YearMonth')
# convert YearMonth back to a datetime for plotting if needed
res_q4['YearMonth'] = res_q4['YearMonth'].dt.to_timestamp()
res_q4


,YearMonth,Total Sales
0,2020-01-01,16253746.0
1,2020-02-01,14997988.0
2,2020-03-01,17660577.0
3,2020-04-01,24607006.0
4,2020-05-01,16918014.0
5,2020-06-01,8829819.0
6,2020-07-01,17146013.0
7,2020-08-01,19877980.0
8,2020-09-01,18304436.0
9,2020-10-01,10836269.0


**Q5 — Region that contributes the most to total sales (and compare to profit)**

In [8]:
# Q5
region_sales = df.groupby('Region', observed=False)['Total Sales'].sum().reset_index().sort_values('Total Sales', ascending=False)
region_profit = df.groupby('Region', observed=False)['Operating Profit'].sum().reset_index().sort_values('Operating Profit', ascending=False)

display(region_sales.head(10))
display(region_profit.head(10))

# Quick check: does top-sales region match top-profit region?
top_sales_region = region_sales.iloc[0]['Region'] if not region_sales.empty else None
top_profit_region = region_profit.iloc[0]['Region'] if not region_profit.empty else None
top_sales_region, top_profit_region


,Region,Total Sales
4,West,269943182.0
1,Northeast,186324067.0
3,Southeast,163171236.0
2,South,144663181.0
0,Midwest,135800459.0


,Region,Operating Profit
4,West,89609406.55
1,Northeast,68020587.65
2,South,61138004.07
3,Southeast,60555416.70
0,Midwest,52811346.48


('West', 'West')

**Q6 — Dates/weeks with significant spikes or drops in sales (anomalies)**

In [9]:
# Q6
daily = df.groupby(df['Invoice Date'].dt.date, observed=False)['Total Sales'].sum().reset_index().rename(columns={'Invoice Date':'Date','Total Sales':'Sales'})
daily['Pct_Change'] = daily['Sales'].pct_change() * 100

top_spikes = daily.sort_values('Pct_Change', ascending=False).head(10)
top_drops  = daily.sort_values('Pct_Change', ascending=True).head(10)

display(top_spikes)
display(top_drops)

# If you prefer weekly aggregation:
weekly = df.groupby(df['Invoice Date'].dt.to_period('W'), observed=False)['Total Sales'].sum().reset_index()
weekly['WeekStart'] = weekly['Invoice Date'].dt.start_time
weekly = weekly.rename(columns={'Total Sales':'Sales'}).sort_values('WeekStart')
weekly.head()


,Date,Sales,Pct_Change
526,2021-06-17,10239903.0,8954.809529
711,2021-12-19,3859805.0,7216.888459
694,2021-12-02,4579102.0,3241.434618
700,2021-12-08,3780923.0,3076.178595
544,2021-07-05,4594907.0,2575.844699
665,2021-11-03,3353526.0,2413.793336
671,2021-11-09,3231871.0,2061.352906
688,2021-11-26,2196503.0,2047.875108
707,2021-12-15,2468339.0,1636.469289
657,2021-10-26,1620128.0,1470.500194


,Date,Sales,Pct_Change
710,2021-12-18,52752.0,-98.746067
687,2021-11-25,102264.0,-97.357300
716,2021-12-24,190885.0,-96.750155
656,2021-10-25,103160.0,-96.089032
595,2021-08-25,230555.0,-96.034358
563,2021-07-24,312736.0,-95.932754
534,2021-06-25,279527.0,-95.651884
525,2021-06-16,113088.0,-95.450916
706,2021-12-14,142147.0,-95.410834
699,2021-12-07,119040.0,-95.162071


,Invoice Date,Sales,WeekStart
0,2019-12-30/2020-01-05,3490033.0,2019-12-30
1,2020-01-06/2020-01-12,2837846.0,2020-01-06
2,2020-01-13/2020-01-19,1410638.0,2020-01-13
3,2020-01-20/2020-01-26,4831822.0,2020-01-20
4,2020-01-27/2020-02-02,5353189.0,2020-01-27


**Q7 — Which product category has the highest operating profit across both sales methods**

In [10]:
# Q7
res_q7 = df.groupby(['Product Category','Sales Method'], observed=False)['Operating Profit'].sum().reset_index().sort_values('Operating Profit', ascending=False)
res_q7.head(20)


,Product Category,Sales Method,Operating Profit
6,Men's Street Footwear,In-store,34867100.00
9,Women's Apparel,In-store,26285125.00
8,Men's Street Footwear,Outlet,24977138.86
7,Men's Street Footwear,Online,22958021.76
11,Women's Apparel,Outlet,21522246.23
10,Women's Apparel,Online,20843599.33
3,Men's Athletic Footwear,In-store,19702375.00
0,Men's Apparel,In-store,17474500.00
17,Women's Street Footwear,Outlet,16690953.56
5,Men's Athletic Footwear,Outlet,16470335.40


**Q8 — Average profit margin for In-store vs Outlet methods**

In [11]:
# Q8
# Compute profit margin per row first (Operating Profit / Total Sales * 100), handle division by zero
df['Profit Margin %'] = np.where(df['Total Sales'] != 0, df['Operating Profit'] / df['Total Sales'] * 100, np.nan)

res_q8 = df.groupby('Sales Method', observed=False)['Profit Margin %'].mean().reset_index().sort_values('Profit Margin %', ascending=False)
res_q8


,Sales Method,Profit Margin %
1,Online,46.412114
2,Outlet,39.487239
0,In-store,35.612069


**Q9 — Which product had the lowest operating margin (and how to inspect it)**

In [12]:
# Q9
# Show bottom 10 products by Operating Margin (product-level aggregation)
prod_margin = df.groupby('Product', observed=False)['Operating Margin'].mean().reset_index().sort_values('Operating Margin', ascending=True)
prod_margin.head(10)


,Product,Operating Margin
1,Men's Athletic Footwear,0.402702
5,Women's Street Footwear,0.410199
0,Men's Apparel,0.413225
4,Women's Athletic Footwear,0.424359
3,Women's Apparel,0.441318
2,Men's Street Footwear,0.446130


**Q10 — Total revenue from each product in the Men's Footwear category**

In [13]:
# Q10
# Filter Product Category text for "Men" (case-insensitive). Adjust pattern if categories are named exactly "Men's Footwear"
men_mask = df['Product Category'].str.contains(r'\bMen', case=False, na=False)
men_footwear = df[men_mask].copy()

res_q10 = men_footwear.groupby('Product', observed=False)['Total Sales'].sum().reset_index().sort_values('Total Sales', ascending=False)
res_q10


,Product,Total Sales
2,Men's Street Footwear,208826244.0
1,Men's Athletic Footwear,153673680.0
0,Men's Apparel,123728632.0


**Q11 — Compare Women's Athletic Footwear vs Women's Street Footwear (sales & profit)**

In [14]:
# Q11
# Adjust these filters if your Product Category uses slightly different labels
athletic_mask = df['Product Category'].str.contains('Women.*Athletic', case=False, na=False) | df['Product'].str.contains('Women.*Athletic', case=False, na=False)
street_mask   = df['Product Category'].str.contains('Women.*Street', case=False, na=False)   | df['Product'].str.contains('Women.*Street', case=False, na=False)

athletic = df[athletic_mask].groupby('Product Category', observed=False)[['Total Sales','Operating Profit']].sum().reset_index()
street   = df[street_mask].groupby('Product Category', observed=False)[['Total Sales','Operating Profit']].sum().reset_index()

display(athletic, street)

# Combined view
combined = pd.concat([athletic, street], ignore_index=True).fillna(0)
combined


,Product Category,Total Sales,Operating Profit
0,Women's Athletic Footwear,106631896.0,38975784.94


,Product Category,Total Sales,Operating Profit
0,Women's Street Footwear,128002813.0,45095826.81


,Product Category,Total Sales,Operating Profit
0,Women's Athletic Footwear,106631896.0,38975784.94
1,Women's Street Footwear,128002813.0,45095826.81


**Q12 — Average units sold per day for each product category**

In [15]:
# Q12
df['Day'] = df['Invoice Date'].dt.date
units_per_day = df.groupby(['Product Category','Day'], observed=False)['Units Sold'].sum().reset_index()
avg_units_per_day = units_per_day.groupby('Product Category', observed=False)['Units Sold'].mean().reset_index().rename(columns={'Units Sold':'Avg Units/Day'}).sort_values('Avg Units/Day', ascending=False)
avg_units_per_day


,Product Category,Avg Units/Day
2,Men's Street Footwear,1630.000000
1,Men's Athletic Footwear,1223.387640
3,Women's Apparel,1222.047887
5,Women's Street Footwear,1080.630854
4,Women's Athletic Footwear,891.112360
0,Men's Apparel,842.535714
